In [2]:
import numpy as np
import matplotlib.pyplot as plt
import urllib
from matplotlib import image

from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import h5py as h5py
import pickle
import os, cv2
from PIL import Image



%matplotlib inline

# Load Data

In [19]:
img_urls = np.loadtxt('faces.csv', dtype=np.str)

In [20]:
def load_images_from_urls(urls):
    for i in range(len(urls)):
        try:
            urllib.request.urlretrieve(urls[i], 'images/img' + str(i) +'.png' )
        except:
            continue

# Try Basic-Yolo-Keras implementation

In [3]:
anchors = np.fromfile('tiny-yolo-keras/tiny-yolo_anchors.txt', sep=', ')
IMAGE_W = 416
CHANNELS = 3
LABELS = 80
GRID = 13
BOX = LABELS+5
ANCHORS = len(anchors)

In [36]:
x_input = Input((IMAGE_W, IMAGE_W, CHANNELS))

#1
x = Conv2D(kernel_size=(3,3), filters=16, strides=(1,1), padding='SAME', name='conv2d_1', use_bias=False)(x_input)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#2
x = Conv2D(kernel_size=(3,3), filters=32, strides=(1,1), padding='SAME', name='conv2d_2', use_bias=False)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#3
x = Conv2D(kernel_size=(3,3), filters=64, strides=(1,1), padding='SAME', name='conv2d_3', use_bias=False)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#4
x = Conv2D(kernel_size=(3,3), filters=128, strides=(1,1), padding='SAME', name='conv2d_4', use_bias=False)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#5
x = Conv2D(kernel_size=(3,3), filters=256, strides=(1,1), padding='SAME', name='conv2d_5', use_bias=False)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#6
x = Conv2D(kernel_size=(3,3), filters=512, strides=(1,1), padding='SAME', name='conv2d_6', use_bias=False)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = MaxPooling2D((2,2))(x)

#7
x = Conv2D(kernel_size=(3,3), filters=1024, strides=(1,1), padding='SAME', name='conv2d_7', use_bias=False)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

#######

x = Conv2D(kernel_size=(3,3), filters=512, strides=(1,1), padding='SAME', name='conv2d_8', use_bias=False)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(kernel_size=(1,1), filters=425, strides=(1,1), padding='SAME', name='conv2d_9')(x)

model = Model(inputs=x_input, outputs=x)

In [37]:
model.load_weights('tiny-yolo-keras/tiny-yolo.h5')

In [39]:
image = plt.imread('street.jpg')
resized = cv2.resize(image, (IMAGE_W, IMAGE_W))
image_data = np.expand_dims(resized, axis=0)
image_data = image_data / 255.
model.predict(image_data)

array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ...

In [46]:
def netout_to_boxes(netout, anchors):
    box_confidences = tf.Variable([])
    box_wh = []
    box_xy = []
    box_class_probs = []
    for row in range(GRID):
        for col in range(GRID):
            for b in ANCHORS:
                netout_box = netout[row,col,0+BOX*b:5+BOX*b]
                n

array([8.99612669e-096, 5.64698265e-038, 3.69907257e-057, 1.10711148e-047,
       3.67057138e-062, 4.24787601e-086, 3.62603836e-043, 1.05148322e-153,
       1.13370004e-095, 7.85919304e-067, 8.98061418e-067])